In [2]:
import pandas as pd

# Load the data
artworks_df = pd.read_csv('../temporary-files/artsoul_artworks_info.csv')
artworks_df

,url,Price,Artist,Artist_url,Title,Description,Height,Width,Depth,Location,Year,Techniques,Topics,Colours
0,https://en.artsoul.com.br/obras/mosca,1200.0,Marcia C. Cavalcanti,https://en.artsoul.com.br/artistas/marcia-c-ca...,Fly,Painting - oil on canvas,Figurativo,Pop,Moderno,RIO DE JANEIRO,2018.0,['Painting'],"['Contemporâneo', 'Figurativo', 'Pop', 'Modern...",['Colorido']
1,https://en.artsoul.com.br/obras/sem-titulo-1501,3500.0,Fabricio Lopez,https://en.artsoul.com.br/artistas/fabricio-lopez,Untitled,"sem título, monotipia, PU (prova única), 2007,...",Abstrato,57,53,SÃO PAULO,2007.0,['Engraving'],"['Contemporâneo', 'Abstrato']",['Colorido']
2,https://en.artsoul.com.br/obras/bye-bye-dubai,2300.0,Francisco Maringelli,https://en.artsoul.com.br/artistas/francisco-m...,"Bye, bye Dubai","Francisco Maringelli, xilogravura de fio sobre...",50,70,NaN,SÃO PAULO,2010.0,NaN,NaN,NaN
3,https://en.artsoul.com.br/obras/on-my-way,23000.0,Anna Paola Protasio,https://en.artsoul.com.br/artistas/anna-paola-...,On My Way,Alumínio e grampos,103,144,NaN,SÃO PAULO,2019.0,['Mixed Technique'],['Contemporary'],['Monocromático']
4,https://en.artsoul.com.br/obras/sem-titulo-2320,1950.0,ADRIANA BRZEZINSKA,https://en.artsoul.com.br/artistas/adriana-brz...,Untitled,Mixed media on canvas,70,50,5,CURITIBA,2019.0,['Outra Técnica'],['Contemporary'],NaN
5,https://en.artsoul.com.br/obras/sem-titulo-1847,9500.0,Fernando Cardoso,https://en.artsoul.com.br/artistas/fernando-ca...,Untitled,Óleo sobre tela.\nYouTube: YBAKATU VISITA ATEL...,Figurativo,Contemporâneo,Surrealista,CURITIBA,2021.0,['Design'],"['Retrato', 'Figurativo', 'Contemporâneo', 'Su...",['Colorido']
6,https://en.artsoul.com.br/obras/bicho-2,3200.0,Adriana Affortunati,https://en.artsoul.com.br/artistas/adriana-aff...,Bicho,Objeto de ParedeMaterial: linho e linhaObra ún...,Surrealista,28,20,SÃO PAULO,2022.0,['Sculpture'],"['Contemporâneo', 'Surrealista']",['Monocromático']
7,https://en.artsoul.com.br/obras/serie-genealog...,1800.0,Ana Schieck,https://en.artsoul.com.br/artistas/ana-schieck,Genealogy Series - Ana Juracema,"Digital photo manipulation, laser print on coa...",Figurativo,Retrato,Conceitual,RIO DE JANEIRO,2019.0,['Photography'],"['Contemporâneo', 'Figurativo', 'Retrato', 'Co...",['Monocromático']
8,https://en.artsoul.com.br/obras/serie-o-inatin...,7500.0,Uiara Bartira,https://en.artsoul.com.br/artistas/uiara-barti...,The Unattainable Series,PAINTING - Acrylic on canvas,Abstrato,Outro Tema,70,RIO DE JANEIRO,2019.0,['Painting'],"['Contemporâneo', 'Abstrato', 'Outro Tema']",['Colorido']
9,https://en.artsoul.com.br/obras/jarros,6250.0,Maria Perdigão,https://en.artsoul.com.br/artistas/maria-perdigao,Jars,"Impressão s/ tela - Pigmentos Naturais (Solos,...",Paisagem,Natureza,Outro Tema,RIO DE JANEIRO,2017.0,['Outra Técnica'],"['Contemporâneo', 'Paisagem', 'Natureza', 'Out...",['Colorido']
